In [ ]:
from templateobs import *

# Exercise 2
## Exercise 2.1

In [7]:
def sup(D, X, Y=None):
    supp = 0
    if Y is None:
        for transaction in D:
            if all(item in transaction for item in X):
                supp += 1
    else:
        for transaction in D:
            if all(item in transaction for item in X) and all(item in transaction for item in Y):
                supp += 1
    return supp

def conf(D, X, Y):
    return sup(D, X, Y) / sup(D, X)

def lift(D, X, Y=None):
    if Y is None:
        Y = X.copy()
    supports = [sum(all(item in transaction for item in items) for transaction in D) for items in (X, Y, X + Y)]
    if 0 in (r_support := [sup / len(D) for sup in supports]):
        return None
    return r_support[2] / (r_support[0] * r_support[1])

def leverage(D, X, Y):
    supports = [sum(all(item in transaction for item in items) for transaction in D) for items in (X, Y, X + Y)]
    final_leverage = supports[2] / len(D) - (supports[0] / len(D)) * (supports[1] / len(D))
    return final_leverage

def jaccard(D, X, Y):
    return sup(D, X, Y) / (sup(D, X) + sup(D, Y) - sup(D, X, Y))

def conviction(D, X, Y):
    support_y = sum(all(item in transaction for item in Y) for transaction in D) / len(D)
    confidence_xy = conf(D, X, Y)
    if confidence_xy >= 1:
        return None
    return (1 - support_y) / (1 - confidence_xy)

def oddsRatio(D, X, Y):
    supports = {(False, False): 0, (False, True): 0, (True, False): 0, (True, True): 0}
    for transaction in D:
        supports[(all(item in transaction for item in X), all(item in transaction for item in Y))] += 1
    final_odds_ratio = ((supports[(True, True)] + 1) * (supports[(False, False)] + 1)) / ((supports[(False, True)] + 1) * (supports[(True, False)] + 1))
    return final_odds_ratio

def imp(D, X, Y):
    if len(X) < 2:
        return 0
    conf_xy = conf(D, X, Y)
    w_conf = [conf(D, W, Y) for W in X]
    return conf_xy - max(w_conf) if w_conf else 0

def getRuleMetric(D,X,Y,metric):
    if metric == "sup":
        return sup(D,X,Y)
    elif metric == "conf":
        return conf(D,X,Y)
    elif metric == "lift":
        return lift(D,X,Y)
    elif metric == "leverage":
        return leverage(D,X,Y)
    elif metric == "jaccard":
        return jaccard(D,X,Y)
    elif metric == "conviction":
        return conviction(D,X,Y)
    elif metric == "oddsratio":
        return oddsRatio(D,X,Y)
    elif metric == "imp":
        return imp(D,X,Y)
    else:
        return "metrica invalida"
    


In [9]:
def testMetric(metric, D, X, Y, ex):
    exString = str(ex)
    dotIndex = exString.find(".")
    floatPos = len(exString) - dotIndex - 1 if dotIndex >= 0 else 0
    actOrig = getRuleMetric(D, X, Y, metric)
    try:
        act = np.round(actOrig, floatPos)
    except:
        print("PROBLEM WITH " + str(actOrig))
    print (("OK    " if act == ex else "FAIL  ") + metric + "(" + "".join([str(x) for x in X]) + " -> " + "".join([str(x) for x in Y]) + ")" + (("Expected: " + str(ex) + " but was " + str(act))  if act != ex else ""))

def testMetrics():
    D = read_database("exampleset_numeric.dat")
    testMetric("conf", D, [1], [5], 1)
    testMetric("conf", D, [5], [1], 0.8)
    testMetric("conf", D, [2], [5], 0.83)
    testMetric("conf", D, [5], [2], 1)
    testMetric("conf", D, [5], [2, 3], 0.6)
    testMetric("conf", D, [2, 3], [5], 0.75)
    testMetric("lift", D, [1], [2], 1)
    testMetric("lift", D, [1], [3], 0.75)
    testMetric("lift", D, [1], [5], 1.2)
    testMetric("leverage", D, [1], [5], 0.11)
    testMetric("lift", D, [1, 3, 4], [5], 1.2)
    testMetric("leverage", D, [1, 3, 4], [5], 0.03)
    testMetric("jaccard", D, [1], [5], 0.8)
    testMetric("jaccard", D, [1], [2], 0.67)
    testMetric("jaccard", D, [1], [3], 0.33)
    testMetric("conviction", D, [1], [4,5], 2)
    testMetric("conviction", D, [5], [3], 0.83)
    testMetric("conviction", D, [3], [5], 0.67)
    testMetric("conviction", D, [4, 5], [1], np.inf)
    testMetric("oddsratio", D, [1], [3], 0)
    testMetric("oddsratio", D, [1], [4], 3)
    testMetric("imp", D, [2, 5], [3], -0.07)
    testMetric("imp", D, [2, 3], [5], -0.08)
    testMetric("imp", D, [3, 5], [2], 0.0)
    testMetric("imp", D, [4, 5], [1], 0.2)

testMetrics()

FileNotFoundError: [Errno 2] No such file or directory: 'exampleset_numeric.dat'

## Exercise 2.2

In [ ]:
def filterProductiveRules(D, R):
    productive_rules = []
    for rule in R:
        X, Y, sup_XY, conf_XY = rule
        if conf_XY > 0:
            productive_rules.append(rule)
    return productive_rules

In [ ]:
def test_set_equalness(exp, act):
    length_exp = len(exp)
    length_act = len(act)
    print("Length of collection: " + str("OK" if length_exp == length_act else "FAILED, expected length " + str(length_exp) + " but saw " + str(length_act)))
    obsolete = [f for f in act if not f in exp]
    missing = [f for f in exp if not f in act]
    if obsolete:
        print("Found unexpected entries: " + str(obsolete))
        return False
    if missing:
        print("Missing entries: " + str(missing))
        return False
    return True
        
def testFilter():
    D = read_database("exampleset_numeric.dat")
    R = getStrongRules(D, 3, 1.0)
    RF_act = filterProductiveRules(D, R)
    
    for r in RF_act:
        if type(r) != tuple:
            print("Element in reduced rule set is not a tuple: " + str(r))
            return
        if len(r) != 4:
            print("Length of tuple is not 4: " + str(r))
            return
    
    RF_exp = [([4, 5], [1, 2], 3, 1.0), ([1], [2, 5], 4, 1.0), ([4, 5], [1], 3, 1.0), ([1], [5], 4, 1.0)]
    equal = test_set_equalness(RF_exp, RF_act)
    print("Filter test: " + ("OK" if equal else "FAILED"))
    
testFilter()

## Exercise 2.3
no tests for this exercise